## Benders Example

In [34]:
import linopy as lp
import gurobipy as gp
from gurobipy import GRB
import logging

#### Define and solve a energy planning model for one year and one time step 

In [35]:
# Create the model
original_model = lp.Model()

# Adding variables
cost_total = original_model.add_variables(name="cost_total", lower=0)

cost_capex_total = original_model.add_variables(name="cost_capex_total", lower=0)
cost_opex_total = original_model.add_variables(name="cost_opex_total", lower=0)

cost_capex_production_boiler = original_model.add_variables(name="cost_capex_production_boiler", lower=0)
cost_opex_natural_gas = original_model.add_variables(name="cost_opex_natural_gas", lower=0)

cost_capex_production_hpp = original_model.add_variables(name="cost_capex_production_hpp", lower=0)
cost_opex_carbon = original_model.add_variables(name="cost_opex_carbon", lower=0)

capacity_boiler = original_model.add_variables(name="capacity_boiler", lower=0, upper=500)
capacity_hpp = original_model.add_variables(name="capacity_hpp", lower=0, upper=1000)

flow_natual_gas = original_model.add_variables(name="flow_natual_gas", lower=0)
flow_carbon = original_model.add_variables(name="flow_carbon", lower=0)

# Parameters
cost_capex_production = {"boiler": 90, "hpp": 100}
cost_opex_production = {"natural_gas": 10, "carbon": 5}
cost_carrier = {"natural_gas": 1, "carbon": 2}
demand = 1000


# Adding constraints
constraint_cost_total = original_model.add_constraints(cost_total == cost_capex_total + cost_opex_total, name="constraint_cost_total")
constraint_cost_capex_total = original_model.add_constraints(cost_capex_total == cost_capex_production_boiler + cost_capex_production_hpp, name="constraint_cost_capex_total")
constraint_cost_opex_total = original_model.add_constraints(cost_opex_total == cost_opex_natural_gas + cost_opex_carbon, name="constraint_cost_opex_total")

constraint_cost_capex_production_boiler = original_model.add_constraints(cost_capex_production_boiler == cost_capex_production["boiler"] * capacity_boiler, name="cost_capex_production_boiler")
constraint_cost_capex_production_hpp = original_model.add_constraints(cost_capex_production_hpp == cost_capex_production["hpp"] * capacity_hpp, name="cost_capex_production_hpp")

constraint_cost_opex_natural_gas = original_model.add_constraints(cost_opex_natural_gas == cost_opex_production["natural_gas"] * flow_natual_gas, name="constraint_cost_opex_production_natural_gas")
constraint_cost_opex_carbon = original_model.add_constraints(cost_opex_carbon == cost_opex_production["carbon"] * flow_carbon, name="constraint_cost_opex_production_carbon")

constraint_carrier_limit_natural_gas = original_model.add_constraints(flow_natual_gas + 50 <= capacity_boiler, name="constraint_carrier_limit_natural_gas")
constraint_carrier_limit_carbon = original_model.add_constraints(flow_carbon +100 <= capacity_hpp, name="constraint_carrier_limit_carbon")

constraint_demand = original_model.add_constraints(flow_carbon + flow_natual_gas == demand, name="constraint_demand")

# Adding objective
capacity_sum = 2 * capacity_boiler + 6 * capacity_hpp
original_model.add_objective(capacity_sum, sense="min")

# Solving the model
original_model.solve()


Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-18
Read LP format model from file /private/var/folders/h1/3dzmgr957wqclq406bh8y4mc0000gn/T/linopy-problem-jv89fnas.lp
Reading time = 0.00 seconds
obj: 10 rows, 11 columns, 23 nonzeros
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 11 columns and 23 nonzeros
Model fingerprint: 0x8f40d77d
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+00, 6e+00]
  Bounds range     [5e+02, 1e+03]
  RHS range        [5e+01, 1e+03]
Presolve removed 10 rows and 11 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 secon

('ok', 'optimal')

In [36]:
original_model.solution

<xarray.Dataset> Size: 88B
Dimensions:                       ()
Data variables:
    cost_total                    float64 8B 1.172e+05
    cost_capex_total              float64 8B 1.1e+05
    cost_opex_total               float64 8B 7.25e+03
    cost_capex_production_boiler  float64 8B 4.5e+04
    cost_opex_natural_gas         float64 8B 4.5e+03
    cost_capex_production_hpp     float64 8B 6.5e+04
    cost_opex_carbon              float64 8B 2.75e+03
    capacity_boiler               float64 8B 500.0
    capacity_hpp                  float64 8B 650.0
    flow_natual_gas               float64 8B 450.0
    flow_carbon                   float64 8B 550.0

In [37]:
gurobi_model = original_model.to_gurobipy()
gurobi_model.write("gurobi_original_model.lp")

In [38]:
# Map variables names in the original model to gurobi variables
list_of_orignal_variables = list(original_model.variables)
list_of_gurobi_variables = gurobi_model.getVars()
mapping_variables_from_original_to_gurobi = {list_of_orignal_variables[i]: list_of_gurobi_variables[i] for i in range(gurobi_model.NumVars)}

# Map constraints names in the original model to gurobi constraints
list_of_orignal_constraints = list(original_model.constraints)
list_of_gurobi_constraints = gurobi_model.getConstrs()
mapping_constraints_from_original_to_gurobi = {list_of_orignal_constraints[i]: list_of_gurobi_constraints[i] for i in range(gurobi_model.NumConstrs)}

In [39]:
# Define a list of design variables and another one of operational variables
design_variables = ["capacity_boiler", "capacity_hpp", "cost_capex_production_boiler", "cost_capex_production_hpp", "cost_capex_total"]
operational_variables = ["flow_natual_gas", "flow_carbon", "cost_opex_natural_gas", "cost_opex_carbon", "cost_opex_total", "cost_total"]

# Define the list of constraints that belong to the design variables and another one for the operational variables
design_constraints = ["constraint_cost_capex_total", "cost_capex_production_boiler", "cost_capex_production_hpp"]
operational_constraints = ["constraint_cost_total", "constraint_cost_opex_total", "constraint_cost_opex_production_natural_gas", "constraint_cost_opex_production_carbon", "constraint_carrier_limit_natural_gas", "constraint_carrier_limit_carbon", "constraint_demand"]

#### DEFINE THE MASTER MODEL IN LINOPY

In [40]:
def define_master_model(operational_variables, operational_constraints):
    """ 
    Define the design problem
    """ 
    master_model = lp.Model()

    # Adding variables
    cost_total = master_model.add_variables(name="cost_total", lower=0)

    cost_capex_total = master_model.add_variables(name="cost_capex_total", lower=0)
    cost_opex_total = master_model.add_variables(name="cost_opex_total", lower=0)

    cost_capex_production_boiler = master_model.add_variables(name="cost_capex_production_boiler", lower=0)
    cost_opex_natural_gas = master_model.add_variables(name="cost_opex_natural_gas", lower=0)

    cost_capex_production_hpp = master_model.add_variables(name="cost_capex_production_hpp", lower=0)
    cost_opex_carbon = master_model.add_variables(name="cost_opex_carbon", lower=0)

    capacity_boiler = master_model.add_variables(name="capacity_boiler", lower=0, upper=500)
    capacity_hpp = master_model.add_variables(name="capacity_hpp", lower=0, upper=1000)

    flow_natual_gas = master_model.add_variables(name="flow_natual_gas", lower=0)
    flow_carbon = master_model.add_variables(name="flow_carbon", lower=0)

    # Parameters
    cost_capex_production = {"boiler": 90, "hpp": 100}
    cost_opex_production = {"natural_gas": 10, "carbon": 5}
    demand = 1000


    # Adding constraints
    constraint_cost_total = master_model.add_constraints(cost_total == cost_capex_total + cost_opex_total, name="constraint_cost_total")
    constraint_cost_capex_total = master_model.add_constraints(cost_capex_total == cost_capex_production_boiler + cost_capex_production_hpp, name="constraint_cost_capex_total")
    constraint_cost_opex_total = master_model.add_constraints(cost_opex_total == cost_opex_natural_gas + cost_opex_carbon, name="constraint_cost_opex_total")

    constraint_cost_capex_production_boiler = master_model.add_constraints(cost_capex_production_boiler == cost_capex_production["boiler"] * capacity_boiler, name="cost_capex_production_boiler")
    constraint_cost_capex_production_hpp = master_model.add_constraints(cost_capex_production_hpp == cost_capex_production["hpp"] * capacity_hpp, name="cost_capex_production_hpp")

    constraint_cost_opex_natural_gas = master_model.add_constraints(cost_opex_natural_gas == cost_opex_production["natural_gas"] * flow_natual_gas, name="constraint_cost_opex_production_natural_gas")
    constraint_cost_opex_carbon = master_model.add_constraints(cost_opex_carbon == cost_opex_production["carbon"] * flow_carbon, name="constraint_cost_opex_production_carbon")

    constraint_carrier_limit_natural_gas = master_model.add_constraints(flow_natual_gas <= capacity_boiler, name="constraint_carrier_limit_natural_gas")
    constraint_carrier_limit_carbon = master_model.add_constraints(flow_carbon <= capacity_hpp, name="constraint_carrier_limit_carbon")

    constraint_demand = master_model.add_constraints(flow_carbon + flow_natual_gas == demand, name="constraint_demand")

    # Adding objective
    capacity_sum = 2 * capacity_boiler + 6 * capacity_hpp
    master_model.add_objective(capacity_sum, sense="min")

    # Remove the operational variables and constraints
    for constraint in operational_constraints:
        master_model.remove_constraints(constraint)

    for variable in operational_variables:
        master_model.remove_variables(variable)

    # Write the model in gurobi format
    master_model_gurobi = master_model.to_gurobipy()
    master_model_gurobi.write("gurobi_master_model.lp")

    # Define again the mapping of variables and constraints
    list_of_master_variables = list(master_model.variables)
    list_of_master_gurobi_variables = master_model_gurobi.getVars()
    mapping_variables_from_master_to_gurobi = {list_of_master_variables[i]: list_of_master_gurobi_variables[i] for i in range(master_model_gurobi.NumVars)}

    list_of_master_constraints = list(master_model.constraints)
    list_of_master_gurobi_constraints = master_model_gurobi.getConstrs()
    mapping_constraints_from_master_to_gurobi = {list_of_master_constraints[i]: list_of_master_gurobi_constraints[i] for i in range(master_model_gurobi.NumConstrs)}

    return master_model, master_model_gurobi, mapping_variables_from_master_to_gurobi, mapping_constraints_from_master_to_gurobi
    
master_model, master_model_gurobi, mapping_variables_from_master_to_gurobi, mapping_constraints_from_master_to_gurobi = define_master_model(operational_variables, operational_constraints)


#### DEFINE THE SUBPROBLEM IN LINOPY   

In [41]:
def define_subproblem_model(design_variables, design_constraints):
    """
    Define the operational problem
    """
    subproblem_model = lp.Model()

    # Adding variables
    cost_total = subproblem_model.add_variables(name="cost_total", lower=0)

    cost_capex_total = subproblem_model.add_variables(name="cost_capex_total", lower=0)
    cost_opex_total = subproblem_model.add_variables(name="cost_opex_total", lower=0)

    cost_capex_production_boiler = subproblem_model.add_variables(name="cost_capex_production_boiler", lower=0)
    cost_opex_natural_gas = subproblem_model.add_variables(name="cost_opex_natural_gas", lower=0)

    cost_capex_production_hpp = subproblem_model.add_variables(name="cost_capex_production_hpp", lower=0)
    cost_opex_carbon = subproblem_model.add_variables(name="cost_opex_carbon", lower=0)

    capacity_boiler = subproblem_model.add_variables(name="capacity_boiler", lower=0, upper=500)
    capacity_hpp = subproblem_model.add_variables(name="capacity_hpp", lower=0, upper=1000)

    flow_natual_gas = subproblem_model.add_variables(name="flow_natual_gas", lower=0)
    flow_carbon = subproblem_model.add_variables(name="flow_carbon", lower=0)

    ## Add a dummy variable fixed to 0 
    dummy = subproblem_model.add_variables(name="dummy", lower=0, upper=0)

    # Parameters
    cost_capex_production = {"boiler": 90, "hpp": 100}
    cost_opex_production = {"natural_gas": 10, "carbon": 5}
    demand = 1000


    # Adding constraints
    constraint_cost_total = subproblem_model.add_constraints(cost_total == cost_capex_total + cost_opex_total, name="constraint_cost_total")
    constraint_cost_capex_total = subproblem_model.add_constraints(cost_capex_total == cost_capex_production_boiler + cost_capex_production_hpp, name="constraint_cost_capex_total")
    constraint_cost_opex_total = subproblem_model.add_constraints(cost_opex_total == cost_opex_natural_gas + cost_opex_carbon, name="constraint_cost_opex_total")

    constraint_cost_capex_production_boiler = subproblem_model.add_constraints(cost_capex_production_boiler == cost_capex_production["boiler"] * capacity_boiler, name="cost_capex_production_boiler")
    constraint_cost_capex_production_hpp = subproblem_model.add_constraints(cost_capex_production_hpp == cost_capex_production["hpp"] * capacity_hpp, name="cost_capex_production_hpp")

    constraint_cost_opex_natural_gas = subproblem_model.add_constraints(cost_opex_natural_gas == cost_opex_production["natural_gas"] * flow_natual_gas, name="constraint_cost_opex_production_natural_gas")
    constraint_cost_opex_carbon = subproblem_model.add_constraints(cost_opex_carbon == cost_opex_production["carbon"] * flow_carbon, name="constraint_cost_opex_production_carbon")

    constraint_carrier_limit_natural_gas = subproblem_model.add_constraints(flow_natual_gas + 50 <= capacity_boiler, name="constraint_carrier_limit_natural_gas")
    constraint_carrier_limit_carbon = subproblem_model.add_constraints(flow_carbon + 100 <= capacity_hpp, name="constraint_carrier_limit_carbon")

    constraint_demand = subproblem_model.add_constraints(flow_carbon + flow_natual_gas == demand, name="constraint_demand")

    # Adding objective
    dummy_objective = dummy.to_linexpr()
    subproblem_model.add_objective(dummy_objective, sense="min", overwrite=True)

    # Remove the operational constraints, the variables are kept to be fixed later on
    for constraint in design_constraints:
        subproblem_model.remove_constraints(constraint)

    # Write the model in gurobi format
    subproblem_model_gurobi = subproblem_model.to_gurobipy()
    subproblem_model_gurobi.write("gurobi_subproblem_model.lp")

    # Define again the mapping of variables and constraints
    list_of_subproblem_variables = list(subproblem_model.variables)
    list_of_subproblem_gurobi_variables = subproblem_model_gurobi.getVars()
    mapping_variables_from_subproblem_to_gurobi = {list_of_subproblem_variables[i]: list_of_subproblem_gurobi_variables[i] for i in range(subproblem_model_gurobi.NumVars)}

    list_of_subproblem_constraints = list(subproblem_model.constraints)
    list_of_subproblem_gurobi_constraints = subproblem_model_gurobi.getConstrs()
    mapping_constraints_from_subproblem_to_gurobi = {list_of_subproblem_constraints[i]: list_of_subproblem_gurobi_constraints[i] for i in range(subproblem_model_gurobi.NumConstrs)}

    return subproblem_model, subproblem_model_gurobi, mapping_variables_from_subproblem_to_gurobi, mapping_constraints_from_subproblem_to_gurobi
    
subproblem_model, subproblem_model_gurobi, mapping_variables_from_subproblem_to_gurobi, mapping_constraints_from_subproblem_to_gurobi = define_subproblem_model(operational_variables, design_constraints)


#### Fix subproblem variables

In [42]:
def fix_design_variables_in_subproblem_model(subproblem_model, master_model):
    """
    Fix the design variables in the subproblem model by adding constraints
    """
    for variable_name in master_model.variables:
        subproblem_model.variables[variable_name].lower = master_model.solution[variable_name].values
        subproblem_model.variables[variable_name].upper = master_model.solution[variable_name].values


    return subproblem_model

In [43]:
def subproblem_to_gurobi(subproblem_solved, iteration):
    """
    Convert the subproblem model to gurobi, set the parameter InfUnbdInfo to 1 and do the mapping of variables and constraints
    """
    subproblem_model_fixed_design_variable_gurobi = subproblem_solved.to_gurobipy()
    subproblem_model_fixed_design_variable_gurobi.write(f"gurobi_subproblem_model_fixed_design_variable_{iteration}.lp")
    subproblem_model_fixed_design_variable_gurobi.setParam(GRB.Param.InfUnbdInfo, 1)

    list_of_subproblem_variables_fixed_design_variable = list(subproblem_solved.variables)
    list_of_subproblem_gurobi_variables_fixed_design_variable = subproblem_model_fixed_design_variable_gurobi.getVars()
    mapping_variables_from_subproblem_to_gurobi_fixed_design_variable = {list_of_subproblem_variables_fixed_design_variable[i]: list_of_subproblem_gurobi_variables_fixed_design_variable[i] for i in range(subproblem_model_fixed_design_variable_gurobi.NumVars)}

    list_of_subproblem_constraints_fixed_design_variable = list(subproblem_solved.constraints)
    list_of_subproblem_gurobi_constraints_fixed_design_variable = subproblem_model_fixed_design_variable_gurobi.getConstrs()
    mapping_constraints_from_subproblem_to_gurobi_fixed_design_variable = {list_of_subproblem_constraints_fixed_design_variable[i]: list_of_subproblem_gurobi_constraints_fixed_design_variable[i] for i in range(subproblem_model_fixed_design_variable_gurobi.NumConstrs)}

    return subproblem_model_fixed_design_variable_gurobi, mapping_variables_from_subproblem_to_gurobi_fixed_design_variable, mapping_constraints_from_subproblem_to_gurobi_fixed_design_variable

#### Solve the Master problem

In [44]:
master_model.solve()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-18
Read LP format model from file /private/var/folders/h1/3dzmgr957wqclq406bh8y4mc0000gn/T/linopy-problem-cv_u1q96.lp
Reading time = 0.00 seconds
obj: 3 rows, 5 columns, 7 nonzeros
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 5 columns and 7 nonzeros
Model fingerprint: 0x9723f6f4
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+00, 6e+00]
  Bounds range     [5e+02, 1e+03]
  RHS range        [0e+00, 0e+00]
Presolve removed 3 rows and 5 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00

('ok', 'optimal')

In [46]:
# Print variables values
master_model.solution

<xarray.Dataset> Size: 40B
Dimensions:                       ()
Data variables:
    cost_capex_total              float64 8B 0.0
    cost_capex_production_boiler  float64 8B 0.0
    cost_capex_production_hpp     float64 8B 0.0
    capacity_boiler               float64 8B 0.0
    capacity_hpp                  float64 8B 0.0

#### Solve the Subproblem with current master solution

In [47]:
def fix_design_variables_in_subproblem_model(subproblem_model, master_model):
    """
    Fix the design variables in the subproblem model by adding constraints
    """
    for variable_name in master_model.variables:
        subproblem_model.variables[variable_name].lower = master_model.solution[variable_name].values
        subproblem_model.variables[variable_name].upper = master_model.solution[variable_name].values


    return subproblem_model
subproblem_model = fix_design_variables_in_subproblem_model(subproblem_model=subproblem_model, master_model=master_model)


In [49]:
subproblem_model.solve(io_api='direct')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-18
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 12 columns and 16 nonzeros
Model fingerprint: 0x673a7aac
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 1e+03]
Presolve removed 4 rows and 10 columns
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Infeasible model


Status: warning
Termination condition: infeasible
Solution: 0 primals, 0 duals
Objective: nan
Solver model: available
Solver message: 3



('warning', 'infeasible')

In [50]:
subproblem_model.print_infeasibilities()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads


IIS computed: 1 constraints and 2 bounds
IIS runtime: 0.00 seconds (0.00 work units)
constraint_carrier_limit_natural_gas: +1 flow_natual_gas - 1 capacity_boiler ≤ -50


#### Add feasibility cut

In [51]:
# Define a function to update the gurobi model of the subproblem, set the Parameter InfUnbdInfo to 1, and do again the mapping of variables and constraints
def subproblem_to_gurobi(subproblem_solved, iteration):
    """
    Convert the subproblem model to gurobi, set the parameter InfUnbdInfo to 1 and do the mapping of variables and constraints
    """
    subproblem_model_fixed_design_variable_gurobi = subproblem_solved.to_gurobipy()
    subproblem_model_fixed_design_variable_gurobi.write(f"gurobi_subproblem_model_fixed_design_variable_{iteration}.lp")
    subproblem_model_fixed_design_variable_gurobi.setParam(GRB.Param.InfUnbdInfo, 1)

    list_of_subproblem_variables_fixed_design_variable = list(subproblem_solved.variables)
    list_of_subproblem_gurobi_variables_fixed_design_variable = subproblem_model_fixed_design_variable_gurobi.getVars()
    mapping_variables_from_subproblem_to_gurobi_fixed_design_variable = {list_of_subproblem_variables_fixed_design_variable[i]: list_of_subproblem_gurobi_variables_fixed_design_variable[i] for i in range(subproblem_model_fixed_design_variable_gurobi.NumVars)}

    list_of_subproblem_constraints_fixed_design_variable = list(subproblem_solved.constraints)
    list_of_subproblem_gurobi_constraints_fixed_design_variable = subproblem_model_fixed_design_variable_gurobi.getConstrs()
    mapping_constraints_from_subproblem_to_gurobi_fixed_design_variable = {list_of_subproblem_constraints_fixed_design_variable[i]: list_of_subproblem_gurobi_constraints_fixed_design_variable[i] for i in range(subproblem_model_fixed_design_variable_gurobi.NumConstrs)}

    return subproblem_model_fixed_design_variable_gurobi, mapping_variables_from_subproblem_to_gurobi_fixed_design_variable, mapping_constraints_from_subproblem_to_gurobi_fixed_design_variable

subproblem_model_fixed_design_variable_gurobi_1, mapping_variables_from_subproblem_to_gurobi_fixed_design_variable, mapping_constraints_from_subproblem_to_gurobi_fixed_design_variable = subproblem_to_gurobi(subproblem_solved=subproblem_model, iteration=1)


Set parameter InfUnbdInfo to value 1


In [52]:
subproblem_model_fixed_design_variable_gurobi_1.optimize()
subproblem_model_fixed_design_variable_gurobi_1.computeIIS()
infeasible_constraints = []
for constraint in subproblem_model_fixed_design_variable_gurobi_1.getConstrs():
    if constraint.IISConstr:
        # I want the name of the constraint in the subproblem model, I will use the map: constraint.ConstrName is the item, I need to print the corresponding key
        for key, value in mapping_constraints_from_subproblem_to_gurobi_fixed_design_variable.items():
            if value == constraint:
                infeasible_constraints.append(key)
                print(f"The following constraint is infeasible: {key}")


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 12 columns and 16 nonzeros
Model fingerprint: 0x673a7aac
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]


  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 1e+03]
Presolve removed 4 rows and 10 columns
Presolve time: 0.01s
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.400000e+03   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Infeasible model
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads


IIS computed: 1 constraints and 2 bounds
IIS runtime: 0.00 seconds (0.00 work units)
The following constraint is infeasible: constraint_carrier_limit_natural_gas


In [53]:
# Compute the Farkas multipliers to set the feasibility cut
infeasibility = subproblem_model_fixed_design_variable_gurobi_1.getAttr(GRB.Attr.IISConstr)
farkas_multipliers = subproblem_model_fixed_design_variable_gurobi_1.getAttr(GRB.Attr.FarkasDual)

# Define the feasibility cut
feasibility_cut = 0 
for i, constraint in enumerate(subproblem_model.constraints):
    if infeasibility[i]:
        rhs_constant = subproblem_model.constraints[constraint].rhs.item() * farkas_multipliers[i]
        lhs = subproblem_model.constraints[constraint].lhs 
        linear_expression = - farkas_multipliers[i] * subproblem_model.constraints[constraint].lhs 
        linear_expression += rhs_constant
        feasibility_cut+= linear_expression

feasibility_cut 

LinearExpression
----------------
-1 flow_natual_gas + 1 capacity_boiler - 50

In [54]:
real_feasibility_cut = master_model.variables["capacity_boiler"] - 50
real_feasibility_cut

LinearExpression
----------------
+1 capacity_boiler - 50

#### Add Feasibility cut to master

In [55]:
master_model.add_constraints(real_feasibility_cut >= 0, name="feasibility_cut_1")

Constraint `feasibility_cut_1`
------------------------------
+1 capacity_boiler ≥ 50.0

In [56]:
master_model.constraints

linopy.model.Constraints
------------------------
 * constraint_cost_capex_total
 * cost_capex_production_boiler
 * cost_capex_production_hpp
 * feasibility_cut_1

In [57]:
master_model.solve()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-18
Read LP format model from file /private/var/folders/h1/3dzmgr957wqclq406bh8y4mc0000gn/T/linopy-problem-bujlh40c.lp
Reading time = 0.00 seconds
obj: 4 rows, 5 columns, 8 nonzeros
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 5 columns and 8 nonzeros
Model fingerprint: 0x4f538644
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+00, 6e+00]
  Bounds range     [5e+02, 1e+03]
  RHS range        [5e+01, 5e+01]
Presolve removed 4 rows and 5 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00

('ok', 'optimal')

In [58]:
master_model.solution

<xarray.Dataset> Size: 40B
Dimensions:                       ()
Data variables:
    cost_capex_total              float64 8B 4.5e+03
    cost_capex_production_boiler  float64 8B 4.5e+03
    cost_capex_production_hpp     float64 8B 0.0
    capacity_boiler               float64 8B 50.0
    capacity_hpp                  float64 8B 0.0

#### Second run 

In [59]:
subproblem_model = fix_design_variables_in_subproblem_model(subproblem_model=subproblem_model, master_model=master_model)

In [60]:
subproblem_model.solve(io_api='direct')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-18
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 12 columns and 16 nonzeros
Model fingerprint: 0x19fe6275
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+01, 4e+03]
  RHS range        [5e+01, 1e+03]
Presolve removed 4 rows and 10 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible model


Status: warning
Termination condition: infeasible
Solution: 0 primals, 0 duals
Objective: nan
Solver model: available
Solver message: 3



('warning', 'infeasible')

In [61]:
subproblem_model.print_infeasibilities()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads


IIS computed: 1 constraints and 2 bounds
IIS runtime: 0.00 seconds (0.00 work units)
constraint_carrier_limit_carbon: +1 flow_carbon - 1 capacity_hpp ≤ -100


In [62]:
subproblem_model_fixed_design_variable_gurobi_2, mapping_variables_from_subproblem_to_gurobi_fixed_design_variable, mapping_constraints_from_subproblem_to_gurobi_fixed_design_variable = subproblem_to_gurobi(subproblem_solved=subproblem_model, iteration=2)


Set parameter InfUnbdInfo to value 1


In [63]:
subproblem_model_fixed_design_variable_gurobi_2.optimize()
subproblem_model_fixed_design_variable_gurobi_2.computeIIS()
infeasible_constraints = []
for constraint in subproblem_model_fixed_design_variable_gurobi_2.getConstrs():
    if constraint.IISConstr:
        # I want the name of the constraint in the subproblem model, I will use the map: constraint.ConstrName is the item, I need to print the corresponding key
        for key, value in mapping_constraints_from_subproblem_to_gurobi_fixed_design_variable.items():
            if value == constraint:
                infeasible_constraints.append(key)
                print(f"The following constraint is infeasible: {key}")


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 12 columns and 16 nonzeros
Model fingerprint: 0x19fe6275
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+01, 4e+03]
  RHS range        [5e+01, 1e+03]
Presolve removed 4 rows and 10 columns
Presolve time: 0.02s
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   8.500000e+02   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds (0.00 work units)
Infeasible model
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads


IIS computed: 1 constraints and 2 bounds
IIS runtime: 0.00 s

In [64]:
# Compute the Farkas multipliers to set the feasibility cut
infeasibility = subproblem_model_fixed_design_variable_gurobi_2.getAttr(GRB.Attr.IISConstr)
farkas_multipliers = subproblem_model_fixed_design_variable_gurobi_2.getAttr(GRB.Attr.FarkasDual)

# Define the feasibility cut
feasibility_cut = 0 
for i, constraint in enumerate(subproblem_model.constraints):
    if infeasibility[i]:
        rhs_constant = subproblem_model.constraints[constraint].rhs.item() * farkas_multipliers[i]
        lhs = subproblem_model.constraints[constraint].lhs 
        linear_expression = - farkas_multipliers[i] * subproblem_model.constraints[constraint].lhs 
        linear_expression += rhs_constant
        feasibility_cut+= linear_expression

feasibility_cut 

LinearExpression
----------------
-1 flow_carbon + 1 capacity_hpp - 100

In [65]:
real_feasibility_cut = master_model.variables["capacity_hpp"] - 100
real_feasibility_cut

LinearExpression
----------------
+1 capacity_hpp - 100

In [66]:
master_model.add_constraints(real_feasibility_cut >= 0, name="feasibility_cut_2")

Constraint `feasibility_cut_2`
------------------------------
+1 capacity_hpp ≥ 100.0

In [67]:
master_model.solve()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-18
Read LP format model from file /private/var/folders/h1/3dzmgr957wqclq406bh8y4mc0000gn/T/linopy-problem-ll7jovnk.lp
Reading time = 0.00 seconds
obj: 5 rows, 5 columns, 9 nonzeros
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 5 columns and 9 nonzeros
Model fingerprint: 0x3a6b3b15
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+00, 6e+00]
  Bounds range     [5e+02, 1e+03]
  RHS range        [5e+01, 1e+02]
Presolve removed 5 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00

('ok', 'optimal')

In [68]:
master_model.solution

<xarray.Dataset> Size: 40B
Dimensions:                       ()
Data variables:
    cost_capex_total              float64 8B 1.45e+04
    cost_capex_production_boiler  float64 8B 4.5e+03
    cost_capex_production_hpp     float64 8B 1e+04
    capacity_boiler               float64 8B 50.0
    capacity_hpp                  float64 8B 100.0

#### Third run

In [69]:
subproblem_model = fix_design_variables_in_subproblem_model(subproblem_model=subproblem_model, master_model=master_model)
subproblem_model.solve(io_api='direct')
subproblem_model.print_infeasibilities()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-18
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 12 columns and 16 nonzeros
Model fingerprint: 0x0d3498c3
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+01, 1e+04]
  RHS range        [5e+01, 1e+03]
Presolve removed 6 rows and 12 columns
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Infeasible model


Status: warning
Termination condition: infeasible
Solution: 0 primals, 0 duals
Objective: nan
Solver model: available
Solver message: 3



Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.500000e+02   0.000000e+00      0s

IIS computed: 3 constraints and 2 bounds
IIS runtime: 0.00 seconds (0.00 work units)
constraint_carrier_limit_natural_gas: +1 flow_natual_gas - 1 capacity_boiler ≤ -50
constraint_carrier_limit_carbon: +1 flow_carbon - 1 capacity_hpp ≤ -100
constraint_demand: +1 flow_carbon + 1 flow_natual_gas = 1000


In [70]:
subproblem_model_fixed_design_variable_gurobi_3, mapping_variables_from_subproblem_to_gurobi_fixed_design_variable, mapping_constraints_from_subproblem_to_gurobi_fixed_design_variable = subproblem_to_gurobi(subproblem_solved=subproblem_model, iteration=3)

Set parameter InfUnbdInfo to value 1


In [71]:
subproblem_model_fixed_design_variable_gurobi_3.optimize()
subproblem_model_fixed_design_variable_gurobi_3.computeIIS()
infeasible_constraints = []
for constraint in subproblem_model_fixed_design_variable_gurobi_3.getConstrs():
    if constraint.IISConstr:
        # I want the name of the constraint in the subproblem model, I will use the map: constraint.ConstrName is the item, I need to print the corresponding key
        for key, value in mapping_constraints_from_subproblem_to_gurobi_fixed_design_variable.items():
            if value == constraint:
                infeasible_constraints.append(key)
                print(f"The following constraint is infeasible: {key}")


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz


Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 12 columns and 16 nonzeros
Model fingerprint: 0x0d3498c3
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+01, 1e+04]
  RHS range        [5e+01, 1e+03]
Presolve removed 6 rows and 12 columns
Presolve time: 0.01s
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.500000e+02   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Infeasible model
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads


IIS computed: 3 constraints and 2 bounds
IIS runtime: 0.00 seconds (0.00 work units)
The following constraint is infeasible: constraint_carrier_limit_natural_gas
The following constraint is infeasi

In [72]:
# Compute the Farkas multipliers to set the feasibility cut
infeasibility = subproblem_model_fixed_design_variable_gurobi_3.getAttr(GRB.Attr.IISConstr)
farkas_multipliers = subproblem_model_fixed_design_variable_gurobi_3.getAttr(GRB.Attr.FarkasDual)

# Define the feasibility cut
feasibility_cut = 0 
for i, constraint in enumerate(subproblem_model.constraints):
    if infeasibility[i]:
        rhs_constant = subproblem_model.constraints[constraint].rhs.item() * farkas_multipliers[i]
        lhs = subproblem_model.constraints[constraint].lhs 
        linear_expression = - farkas_multipliers[i] * subproblem_model.constraints[constraint].lhs 
        linear_expression += rhs_constant
        feasibility_cut+= linear_expression

feasibility_cut 

LinearExpression
----------------
-1 flow_natual_gas + 1 capacity_boiler - 1 flow_carbon + 1 capacity_hpp + 1 flow_carbon + 1 flow_natual_gas - 1150

In [73]:
real_feasibility_cut = master_model.variables["capacity_boiler"] + master_model.variables["capacity_hpp"] - 1150
real_feasibility_cut

LinearExpression
----------------
+1 capacity_boiler + 1 capacity_hpp - 1150

In [74]:
master_model.add_constraints(real_feasibility_cut >= 0, name="feasibility_cut_3")

Constraint `feasibility_cut_3`
------------------------------
+1 capacity_boiler + 1 capacity_hpp ≥ 1150.0

In [75]:
master_model.constraints

linopy.model.Constraints
------------------------
 * constraint_cost_capex_total
 * cost_capex_production_boiler
 * cost_capex_production_hpp
 * feasibility_cut_1
 * feasibility_cut_2
 * feasibility_cut_3

In [76]:
master_model.solve()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-18
Read LP format model from file /private/var/folders/h1/3dzmgr957wqclq406bh8y4mc0000gn/T/linopy-problem-7vg189fv.lp
Reading time = 0.00 seconds
obj: 6 rows, 5 columns, 11 nonzeros
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 5 columns and 11 nonzeros
Model fingerprint: 0x8c8ea778
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+00, 6e+00]
  Bounds range     [5e+02, 1e+03]
  RHS range        [5e+01, 1e+03]
Presolve removed 6 rows and 5 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.9000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.

('ok', 'optimal')

In [77]:
master_model.solution

<xarray.Dataset> Size: 40B
Dimensions:                       ()
Data variables:
    cost_capex_total              float64 8B 1.1e+05
    cost_capex_production_boiler  float64 8B 4.5e+04
    cost_capex_production_hpp     float64 8B 6.5e+04
    capacity_boiler               float64 8B 500.0
    capacity_hpp                  float64 8B 650.0

In [78]:
subproblem_model = fix_design_variables_in_subproblem_model(subproblem_model=subproblem_model, master_model=master_model)
subproblem_model.solve(io_api='direct')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-18
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-8259U CPU @ 2.30GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 12 columns and 16 nonzeros
Model fingerprint: 0x25ab9cf3
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+02, 1e+05]
  RHS range        [5e+01, 1e+03]
Presolve removed 7 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  0.000000000e+00


('ok', 'optimal')

In [79]:
subproblem_model.solution

<xarray.Dataset> Size: 96B
Dimensions:                       ()
Data variables:
    cost_total                    float64 8B 1.172e+05
    cost_capex_total              float64 8B 1.1e+05
    cost_opex_total               float64 8B 7.25e+03
    cost_capex_production_boiler  float64 8B 4.5e+04
    cost_opex_natural_gas         float64 8B 4.5e+03
    cost_capex_production_hpp     float64 8B 6.5e+04
    cost_opex_carbon              float64 8B 2.75e+03
    capacity_boiler               float64 8B 500.0
    capacity_hpp                  float64 8B 650.0
    flow_natual_gas               float64 8B 450.0
    flow_carbon                   float64 8B 550.0
    dummy                         float64 8B 0.0

In [80]:
original_model.solution

<xarray.Dataset> Size: 88B
Dimensions:                       ()
Data variables:
    cost_total                    float64 8B 1.172e+05
    cost_capex_total              float64 8B 1.1e+05
    cost_opex_total               float64 8B 7.25e+03
    cost_capex_production_boiler  float64 8B 4.5e+04
    cost_opex_natural_gas         float64 8B 4.5e+03
    cost_capex_production_hpp     float64 8B 6.5e+04
    cost_opex_carbon              float64 8B 2.75e+03
    capacity_boiler               float64 8B 500.0
    capacity_hpp                  float64 8B 650.0
    flow_natual_gas               float64 8B 450.0
    flow_carbon                   float64 8B 550.0

### OPTIMAL SOLUTION FOUND